# AdaBoost

1. 收集数据：可以使用任意方法。
2. 准备数据：依赖于所使用的弱分类器类型，本章使用的是单层决策树，这种分类器可以处理任何数据类型。当然也可以使用任意分类器作为弱分类器，第2章到第6章中的任一分类器都可以充当弱分类器。作为弱分类器，简单分类器的效果更好。
3. 分析数据：可以使用任意方法。
4. 训练算法： AdaBoost的大部分时间都用在训练上，分类器将多次在同一数据集上训练弱分类器。
5. 测试算法：计算分类的错误率。
6. 使用算法：同SVM一样， AdaBoost预测两个类别中的一个。如果想把它应用到多个类别的场合，那么就要像多类SVM中的做法一样对AdaBoost进行修改。

## 训练算法：基于错误提升分类器的性能

In [1]:
import numpy as np

In [2]:
# 初始化数据
def loadSimpData():
    datMat = np.matrix([[1, 2.1],
                        [2, 1.1],
                        [1.3, 1],
                        [1, 1],
                        [2, 1]])
    classLabels = [1, 1, -1, -1, 1]
    return datMat, classLabels

In [3]:
datMat, classLabels = loadSimpData()

## 构建单层决策树
程序伪代码：  
将最小错误率minError设为$+\infty$  
对数据集中的每一个特征（第一层循环）：  
&emsp;&emsp;对每个步长（第二层循环）：  
&emsp;&emsp;&emsp;&emsp;对每个不等号（第三层循环）：  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;建立一棵单层决策树并利用加权数据集对它进行测试  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;如果错误率低于minError，则将当前单层决策树设为最佳单层决策树  
返回最佳单层决策树

### 单层决策树生成函数

In [4]:
# 通过阈值比较对数据进行分类
def stumpClassify(dataMatrix, dimen, threshVal, threshIneq):
    # 将返回数组中的全部元素设置为1
    retArray = np.ones((np.shape(dataMatrix)[0], 1))
    # 所有满足不等式要求的元素设置为-1
    if threshIneq == 'lt':
        retArray[dataMatrix[:, dimen] <= threshVal] = -1
    else:
        retArray[dataMatrix[:, dimen] > threshVal] = -1
    return retArray

In [5]:
# 遍历所有的可能输入值，并找到数据集上最佳的单层决策树
def buildStump(dataArr, classLabels, D):
    # 将数据矩阵化
    dataMatrix = np.mat(dataArr); labelMat = np.mat(classLabels).T
    m, n = np.shape(dataMatrix)
    # bestStump 用于存储给定权重向量D时所得到的最佳单层决策树的相关信息
    # numSteps 用于在特征的所有可能值上进行遍历
    numSteps = 10; bestStump = {}; bestClasEst =np.mat(np.zeros((m, 1)))
    # 一开始就初始化成正无穷大，用于寻找可能的最小错误率
    minError = np.Inf
    # 在所有特征上遍历
    for i in range(n):
        rangeMin = dataMatrix[:, i].min(); rangeMax = dataMatrix[:, i].max();
        # 计算得到步长
        stepSize = (rangeMax - rangeMin) / numSteps
        # 从最左边开始设置阈值，-1的目的是从一开始就将最小点归为右侧
        for j in range(-1, int(numSteps) + 1):
            for inequal in ['lt', 'gt']:
                # 设置阈值
                threshVal = rangeMin + float(j) * stepSize
                predictedVals = stumpClassify(dataMatrix, i, threshVal, inequal)
                errArr = np.mat(np.ones((m, 1)))
                # 如果predictedVals中的值不等于labelMat中的真正类别标签值，就设置为1
                # 否则设置为0
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T * errArr
                #print("split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError))
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump, minError, bestClasEst

In [6]:
D = np.mat(np.ones((5, 1)) / 5)

In [7]:
buildStump(datMat, classLabels, D)

({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'}, matrix([[0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

## 完整AdaBoost算法

伪代码：  
对每次迭代：  
&emsp;&emsp;利用buildStump()函数找到最佳的单层决策树  
&emsp;&emsp;将最佳单层决策树加入到单层决策树数组  
&emsp;&emsp;计算alpha  
&emsp;&emsp;计算新的权重向量D
&emsp;&emsp;更新累计类别估计值
&emsp;&emsp;如果错误率等于0.0，则退出循环

### 基于单层决策树的AdaBoost训练过程

In [8]:
# dataArr 数据集，classLabels 类别标签, numIt 迭代次数
def adaBoostTrainDS(dataArr, classLabels, numIt=40):
    weakClassArr = []
    m = np.shape(dataArr)[0]
    # D是一个概率分布向量，所有的元素之和为1.0
    D = np.mat(np.ones((m, 1)) / m)
    # 记录每个数据点的类别估计累计值
    aggClassEst = np.mat(np.zeros((m, 1)))
    for i in range(numIt):
        #返回最小错误率的单层决策树，最小错误率，估计的类别向量
        bestStump, error, classEst = buildStump(dataArr, classLabels, D)
        print("D: ", D.T)
        # 计算alpha值
        alpha = float(0.5 * np.log((1.0 - error)/ max(error, 1e-16)))
        bestStump['alpha'] = alpha
        weakClassArr.append(bestStump)
        print("classEst: ", classEst.T)
        # 计算下一次迭代的D
        expon = np.multiply(-1 * alpha * np.mat(classLabels).T, classEst)
        D = np.multiply(D, np.exp(expon))
        D = D / D.sum()
        # 错误率累加计算
        aggClassEst += alpha * classEst
        print("aggClassEst: ", aggClassEst.T)
        aggErrors = np.multiply(np.sign(aggClassEst) != np.mat(classLabels).T, np.ones((m, 1)))
        errorRate = aggErrors.sum() / m
        print("total error: ", errorRate, "\n")
        if errorRate == 0.0: break
    return weakClassArr

In [9]:
adaBoostTrainDS(datMat, classLabels, 9)

D:  [[0.2 0.2 0.2 0.2 0.2]]
classEst:  [[-1.  1. -1. -1.  1.]]
aggClassEst:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error:  0.2 

D:  [[0.5   0.125 0.125 0.125 0.125]]
classEst:  [[ 1.  1. -1. -1. -1.]]
aggClassEst:  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error:  0.2 

D:  [[0.28571429 0.07142857 0.07142857 0.07142857 0.5       ]]
classEst:  [[1. 1. 1. 1. 1.]]
aggClassEst:  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error:  0.0 



[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599453},
 {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276565},
 {'dim': 0, 'thresh': 0.9, 'ineq': 'lt', 'alpha': 0.8958797346140273}]

## 测试算法：基于AdaBoost的分类

In [10]:
# datToClass一个或多个待分类样例，classifierArr多个弱分类器
def adaClassify(datToClass, classifierArr):
    dataMatrix = np.mat(datToClass)
    m = np.shape(dataMatrix)[0]
    aggClassEst = np.mat(np.zeros((m, 1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix, classifierArr[i]['dim'], classifierArr[i]['thresh'], classifierArr[i]['ineq'])
        aggClassEst += classifierArr[i]['alpha'] * classEst
        print(aggClassEst)
    return np.sign(aggClassEst)

In [11]:
datArr, labelArr = loadSimpData()

In [12]:
classifierArr = adaBoostTrainDS(datArr, labelArr, 30)

D:  [[0.2 0.2 0.2 0.2 0.2]]
classEst:  [[-1.  1. -1. -1.  1.]]
aggClassEst:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
total error:  0.2 

D:  [[0.5   0.125 0.125 0.125 0.125]]
classEst:  [[ 1.  1. -1. -1. -1.]]
aggClassEst:  [[ 0.27980789  1.66610226 -1.66610226 -1.66610226 -0.27980789]]
total error:  0.2 

D:  [[0.28571429 0.07142857 0.07142857 0.07142857 0.5       ]]
classEst:  [[1. 1. 1. 1. 1.]]
aggClassEst:  [[ 1.17568763  2.56198199 -0.77022252 -0.77022252  0.61607184]]
total error:  0.0 



In [13]:
adaClassify([0, 0], classifierArr)

[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])

In [14]:
adaClassify([[5, 5], [0, 0]], classifierArr)

[[ 0.69314718]
 [-0.69314718]]
[[ 1.66610226]
 [-1.66610226]]
[[ 2.56198199]
 [-2.56198199]]


matrix([[ 1.],
        [-1.]])